In [1]:
import os
os.environ["ANTHROPIC_API_KEY"]  = ""

In [2]:
!pip install -q langchain-anthropic langchain-community langchain-core requests duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.7/437.7 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.0/264.0 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.8 MB/s eta 0:00:00


In [3]:
from langchain_anthropic import ChatAnthropic
from langchain_core.tools import tool
import requests

In [4]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()


In [17]:
@tool
def get_weather_data(city: str) -> str:
  """
  This function fetches the current weather data for a given city
  """
  url = f'https://api.weatherstack.com/current?access_key=0192ee34e547db2d400a6c2c388e45ee&query={city}'

  response = requests.get(url)

  return response.json()

In [6]:
llm = ChatAnthropic(model="claude-3-7-sonnet-20250219")

In [7]:
llm.invoke("Hii")

AIMessage(content="Hello! How can I assist you today? I'm here to help with any questions or tasks you might have.", additional_kwargs={}, response_metadata={'id': 'msg_01GpCQX95F1YyAt8Zg2Lv3h7', 'model': 'claude-3-7-sonnet-20250219', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 10, 'output_tokens': 26, 'server_tool_use': None}, 'model_name': 'claude-3-7-sonnet-20250219'}, id='run--e5e30aca-0e75-4eda-bb97-8ad6f15d3219-0', usage_metadata={'input_tokens': 10, 'output_tokens': 26, 'total_tokens': 36, 'input_token_details': {'cache_read': 0, 'cache_creation': 0}})

In [8]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub

In [9]:
# Step 2 : pull the ReAct from Langchain Hub

prompt = hub.pull("hwchase17/react")    # pull the standard ReAct agent prompt

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [10]:
# Step 3 : Create the ReAct agent manually with the pulled prompt
agent = create_react_agent(
    llm = llm,
    tools=[search_tool,get_weather_data],
    prompt=prompt
)

In [21]:
# Step 4 : Wrap it with AgentExecuter
agent_executer = AgentExecutor(
    agent = agent,
    tools = [search_tool,get_weather_data],
    max_iterations=10,
    verbose = True
)

In [29]:
# Step 5 : Invoke
response = agent_executer.invoke({"input":"what is the capital of Bihar and what is the current weather condition there"})
print(response)



> Entering new AgentExecutor chain...
Question: what is the capital of Bihar and what is the current weather condition there
Thought: I need to find out two things here:
1. The capital of Bihar (an Indian state)
2. The current weather conditions in that capital city

Let me first determine the capital of Bihar.

Action: duckduckgo_search
Action Input: capital of Bihar India
Bihar, state of eastern India.It is bounded by Nepal to the north and by the Indian states of West Bengal to the northeast and Uttar Pradesh to the west. In November 2000 the new state of Jharkhand was created from Bihar's southern provinces and now forms the state's southern and southeastern borders. The capital of Bihar is Patna.. Bihar occupied an important position in the early history ... Patna, city, capital of Bihar state, northern India. It lies about 290 miles (470 km) northwest of Kolkata (Calcutta). Patna is one of the oldest cities in India. During the Mughal period it was known as Azimabad. Patna is a

In [30]:
response['output']

'Agent stopped due to iteration limit or time limit.'

In [28]:
# Example usage
print(get_weather_data.invoke({"city": "Bhopal"}))

{'request': {'type': 'City', 'query': 'Bhopal, India', 'language': 'en', 'unit': 'm'}, 'location': {'name': 'Bhopal', 'country': 'India', 'region': 'Madhya Pradesh', 'lat': '23.267', 'lon': '77.400', 'timezone_id': 'Asia/Kolkata', 'localtime': '2025-05-10 15:40', 'localtime_epoch': 1746891600, 'utc_offset': '5.50'}, 'current': {'observation_time': '10:10 AM', 'temperature': 35, 'weather_code': 116, 'weather_icons': ['https://cdn.worldweatheronline.com/images/wsymbols01_png_64/wsymbol_0002_sunny_intervals.png'], 'weather_descriptions': ['Partly cloudy'], 'astro': {'sunrise': '05:42 AM', 'sunset': '06:52 PM', 'moonrise': '05:00 PM', 'moonset': '04:00 AM', 'moon_phase': 'Waxing Gibbous', 'moon_illumination': 94}, 'air_quality': {'co': '401.45', 'no2': '2.405', 'o3': '130', 'so2': '9.62', 'pm2_5': '40.33', 'pm10': '76.405', 'us-epa-index': '2', 'gb-defra-index': '2'}, 'wind_speed': 11, 'wind_degree': 237, 'wind_dir': 'WSW', 'pressure': 1008, 'precip': 0, 'humidity': 27, 'cloudcover': 50, '